In [1]:
!ls

AdvertiseGen.tar.gz  homework_lora_finetune.ipynb  README_en.md
configs		     inference_hf.py		   README.md
finetune_hf.py	     lora_finetune.ipynb	   requirements.txt


In [3]:
!gunzip AdvertiseGen.tar.gz

In [5]:
!tar -xvf AdvertiseGen.tar

AdvertiseGen/
AdvertiseGen/train.json
AdvertiseGen/dev.json


In [7]:
import json
from typing import Union
from pathlib import Path


def _resolve_path(path: Union[str, Path]) -> Path:
    return Path(path).expanduser().resolve()


def _mkdir(dir_name: Union[str, Path]):
    dir_name = _resolve_path(dir_name)
    if not dir_name.is_dir():
        dir_name.mkdir(parents=True, exist_ok=False)


def convert_adgen(data_dir: Union[str, Path], save_dir: Union[str, Path]):
    def _convert(in_file: Path, out_file: Path):
        _mkdir(out_file.parent)
        with open(in_file, encoding='utf-8') as fin:
            with open(out_file, 'wt', encoding='utf-8') as fout:
                for line in fin:
                    dct = json.loads(line)
                    sample = {'conversations': [{'role': 'user', 'content': dct['content']},
                                                {'role': 'assistant', 'content': dct['summary']}]}
                    fout.write(json.dumps(sample, ensure_ascii=False) + '\n')

    data_dir = _resolve_path(data_dir)
    save_dir = _resolve_path(save_dir)

    train_file = data_dir / 'train.json'
    if train_file.is_file():
        out_file = save_dir / train_file.relative_to(data_dir)
        _convert(train_file, out_file)

    dev_file = data_dir / 'dev.json'
    if dev_file.is_file():
        out_file = save_dir / dev_file.relative_to(data_dir)
        _convert(dev_file, out_file)


convert_adgen('AdvertiseGen', 'AdvertiseGen_fix')

In [ ]:
conda update -n base -c defaults conda

In [19]:
!conda install gcc_linux-64

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c defaults conda


^C


In [18]:
!pip install -r requirements.txt

DEPRECATION: Loading egg at /data/minconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub-0.22.0rc0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 1.6 MB/s eta 0:00:0000:0100:01
  Using cached https://mirrors.aliyun.com/pypi/packages/b4/5d/758c00ba637bc850f35fff7fad442c470ac3d606fe586d881b0bed7ef5a5/peft-0.10.0-py3-none-any.whl (199 kB)
  Using cached deepspeed-0.13.1-py3-none-any.whl
  Using cached https://mirrors.aliyun.com/pypi/packages/b3/17/1d146e0127b66f1945251f130afac430985d2f9d75a3c0330355f21d876a/mpi4py-3.1.6.tar.gz (2.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
!pip install rouge_chinese

DEPRECATION: Loading egg at /data/minconda3/envs/llm/lib/python3.11/site-packages/huggingface_hub-0.22.0rc0-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/03/0f/394cf877be7b903881020ef7217f7dc644dad158d52a9353fcab22e3464d/rouge_chinese-1.0.3-py3-none-any.whl (21 kB)


In [1]:
!CUDA_VISIBLE_DEVICES=0 python finetune_hf.py  AdvertiseGen_fix  /data/glm/ChatGLM-6B-main/THUDM/chatglm3-6b  configs/lora.yaml

2024-05-14 15:17:11.607770: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 15:17:11.649118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 15:17:12.282106: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:27<00:00,  3.90s/it]
trainable params: 1,949,696 || all params: 6,245,533,696 || trainable%: 0.031217444255383614
--> Model

--> model has 1.949696M params

train_dataset: Dataset({
    feat

In [2]:
!ls output/

checkpoint-100	checkpoint-180	checkpoint-240	checkpoint-40
checkpoint-120	checkpoint-20	checkpoint-260	checkpoint-60
checkpoint-140	checkpoint-200	checkpoint-280	checkpoint-80
checkpoint-160	checkpoint-220	checkpoint-300	runs


In [3]:
!CUDA_VISIBLE_DEVICES=0 python inference_hf.py output/checkpoint-300/ --prompt "类型#裙*版型#显瘦*材质#网纱*风格#性感*裙型#百褶*裙下摆#压褶*裙长#连衣裙*裙衣门襟#拉链*裙衣门襟#套头*裙款式#拼接*裙款式#拉链*裙款式#木耳边*裙款式#抽褶*裙款式#不规则"

Loading checkpoint shards: 100%|██████████████████| 7/7 [00:36<00:00,  5.17s/it]
2024-05-14 15:55:15.705544: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 15:55:16.480593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-14 15:55:17.892474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
百褶连衣裙的款式，加上网纱拼接，让整件连衣裙显得更有层次感。下摆的百褶设计，加上侧边拉链的装饰，显得十分精致。侧边抽褶的装饰，让裙身显得更加修身，加上拼接的木耳边，让整体显得更加优雅。整体的设计，加上网纱的拼接，让连衣裙显得更加性感。
